<a href="https://colab.research.google.com/github/NikolasGialitsis/TopicLearning/blob/master/lda2vec_DNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone  https://github.com/NikolasGialitsis/LDA2vec.git
!mv /content/LDA2vec/lda2vec /content
!rm -rf LDA2vec/

Cloning into 'LDA2vec'...
remote: Enumerating objects: 112, done.
remote: Total 112 (delta 0), reused 0 (delta 0), pack-reused 112
Receiving objects: 100% (112/112), 13.14 MiB | 27.85 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [2]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Setup and Preprocessing

Install Dependencies



In [0]:
#!git clone https://github.com/maxent-ai/lda2vec

!pip install spacy
!pip install jellyfish
!pip install -r /content/lda2vec/requirements.txt 
!pip install pylda2vec


In [0]:
!python -m spacy download en

Import Libraries

In [5]:
%cd /content
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec
from lda2vec import preprocess, Corpus
#changed Preprocess.py line 'nlp = spacy.load("en_core_web_sm")' to solve error en not found
#and removed the import ... as en line 

import logging
logging.basicConfig()
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from lda2vec import preprocess, Corpus

/content


**CUDA** usage\
 *change Notebook Settings 'Hardware Acceleration' to 'GPU' to run*

In [6]:
gpu_id = int(os.getenv('CUDA_GPU', 0))
cuda.get_device(gpu_id).use()
print("Using GPU:" + str(gpu_id))

Using GPU:0


**Fetch Data**\
Set Stopwords
>>**DONE**: replace texts with covid19 collection

In [7]:
filename = '/content/drive/My Drive/FAS_COMBINED.zip (Unzipped Files)/FAS_COMBINED/gor_hg3_content.txt'
data = []
with open(filename, "r") as f:
  data = f.read()
concats = data.split("\n")

for x in concats[:10]:
  print(x)
  


GTGGCT TGGCTG GGCTGC GCTGCA CTGCAC TGCACA GCACAG CACAGG ACAGGC CAGGCT AGGCTC GGCTCC GCTCCT CTCCTC TCCTCC CCTCCC CTCCCC TCCCCC CCCCCT CCCCTG CCCTGT CCTGTG CTGTGC TGTGCC GTGCCA TGCCAA GCCAAA CCAAAA CAAAAA AAAAAT AAAATA AAATAC AATACC ATACCA TACCAC ACCACG CCACGT CACGTC ACGTCC CGTCCC GTCCCT TCCCTG CCCTGT CCTGTG CTGTGG TGTGGC GTGGCC TGGCCA GGCCAC GCCACA CCACAT CACATG ACATGA CATGAG ATGAGA TGAGAC GAGACT AGACTC GACTCG ACTCGT CTCGTG TCGTGC CGTGCC GTGCCA TGCCAG GCCAGC CCAGCC CAGCCA AGCCAT GCCATC CCATCA CATCAG ATCAGG TCAGGG CAGGGT AGGGTC GGGTCC GGTCCA GTCCAG TCCAGG CCAGGC CAGGCC AGGCCA GGCCAC GCCACA CCACAA CACAAA ACAAAT CAAATA AAATAG AATAGA ATAGAT TAGATG AGATGG GATGGC ATGGCA TGGCAT GGCATT GCATTT CATTTG ATTTGA TTTGAG TTGAGG TGAGGG GAGGGC AGGGCC GGGCCC GGCCCT GCCCTC CCCTCC CCTCCA CTCCAG TCCAGC CCAGCT CAGCTA AGCTAC GCTACA CTACAG TACAGT ACAGTA CAGTAC AGTACA GTACAT TACATG ACATGA CATGAG ATGAGG TGAGGA GAGGAC AGGACG GGACGA GACGAC ACGACT CGACTT GACTTC ACTTCC CTTCCT TTCCTG 
CCAGGC CAGGCC AGGCCT GGCCTG GCCTG

Pre-processing

*   Lowercase
*   Remove stopwords from nltk
*   Remove punctuation



In [8]:
import nltk 
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
texts = []
for text in concats.copy():
  word_tokens = word_tokenize(text) 
  #remove stopwords
  text = ' '.join([w.lower() for w in word_tokens if not w.lower() in stop_words and len(w) >= 3]) 
  #remove punctuation
  #bioclean    = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'", '').strip().lower()).split()

  text = ''.join([w for w in text if w not in string.punctuation])
  print(text[:100])
  texts.append(text)
  

Streaming output truncated to the last 5000 lines.
ttctga tctgaa ctgaaa tgaaac gaaacg aaacgc aacgcc acgcct cgcctg gcctgg cctggc ctggct tggctt ggcttc gc
gcactt cacttc acttcc cttccc ttcccg tcccga cccgaa ccgaat cgaatg gaatgt aatgtt atgttt tgtttg gtttgc tt
cttcct ttcctc tcctcc cctccc ctcccc tcccct cccctg ccctgc cctgcc ctgcct tgcctg gcctgt cctgtc ctgtcc tg
ggggag gggagg ggaggg gaggga agggag gggagg ggaggt gaggtc aggtct ggtctg gtctgt tctgtg ctgtgc tgtgcg gt
ggagtt gagttc agttcg gttcgg ttcggt tcggtc cggtcc ggtccg gtccgg tccggg ccgggg cggggc ggggct gggctg gg
agggag gggagg ggaggc gaggcc aggccc ggcccc gccccg ccccga cccgac ccgact cgactg gactgg actggg ctgggc tg
tcaggg cagggt agggtg gggtgg ggtggg gtgggc tgggca gggcag ggcagc gcagca cagcag agcagt gcagtg cagtgg ag
ttacaa tacaag acaagg caaggc aaggca aggcag ggcaga gcagat cagatt agattg gattgt attgtg ttgtgc tgtgcc gt
ccccct cccctg ccctgt cctgtc ctgtcg tgtcgg gtcggc tcggcc cggccg ggccgg gccggc ccggct cggctg ggctga gc
ctgcca tgccaa gccaaa ccaaac caaaca aaaca

Preprocessing


*   set max word num per document
*   remove stopwords (aka. words in 'bad' list above)



In [0]:
sample = np.random.choice(a=texts,size=100,replace=False)

In [10]:
print(sample[:100])

['agttgc gttgca ttgcag tgcaga gcagat cagata agataa gataat ataata taatat aatatg atatgg tatggt atggtg tggtgt ggtgtt gtgttt tgtttt gttttc ttttca tttcag ttcaga tcagat cagatg agatgc gatgct atgctg tgctgg gctggg ctggga tgggag gggagg ggaggt gaggtt aggtta ggttat gttatg ttatga tatgaa atgaaa tgaaac gaaaca aaacat aacata acatat catatt atattt tattta atttaa tttaaa ttaaaa taaaag aaaaga aaagat aagatt agatta gattaa attaag ttaagg taaggc aaggct aggctc ggctct gctctt ctcttt tctttg ctttga tttgac ttgacg tgacgt gacgta acgtaa cgtaag gtaaga taagaa aagaag agaagg gaagga aaggac aggaca ggacaa gacaaa acaaac caaaca aaacag aacaga acagaa cagaag agaagt gaagtg aagtgt agtgtt gtgtta tgttaa gttaaa ttaaat taaata aaatag aatagc atagca tagcac agcact gcacta cactaa actaaa ctaaaa taaaac aaaact aaactt aacttc acttca cttcaa ttcaaa tcaaaa caaaag aaaagc aaagca aagcaa agcaaa gcaaag caaaga aaagat aagatc agatca gatcat atcatt tcattt cattta atttag tttagg ttaggt taggtg aggtgg ggtgga gtggat tggatg ggatgc gatgcc atgcca tgccat gccatg ccatgg catg

In [0]:
# Preprocess data

mean_length = np.mean([len(t) for t in texts])
max_length = int(mean_length)

Calculate tokens (list) and vocabulary (dictionary)\
OR\
Load them if already calculated

In [12]:
'''if a model is already trained,
skip to the cell loading vocab,corpus,flattened and doc_ids'''

skip = -2
vocab = {}
tokens = np.zeros((len(texts), max_length), dtype='int32')
tokens[:] = skip
word_index = {}
c = 0 #id of word in vocabulary
i = 0 #id of instance
for instance in sample:
  loc = 0 #word position in instance
  words = instance.split(' ')
  for word in words:
    index = -1
    if word not in vocab.values():
      vocab[c] = word
      index = c
      c+=1
      word_index[word] = index
    else:
      index = word_index[word]
    tokens[i,loc] = index
    loc += 1
    if loc == max_length:
      break
  i+=1      

vocab[-2] = None 
assert(len(vocab.keys()) <= 4**6 + 1)
#tokens, vocab = preprocess.tokenize(str(sample), max_length, merge=False,n_threads=4)
'''LOAD IF ALREADY COMPUTED'''
#tokens = np.load("tokens.npy")
#vocab = np.load("vocab.npy")


'LOAD IF ALREADY COMPUTED'

In [13]:
print(tokens[10,200:400])

[-2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2 -2
 -2 -2 -2 -2 -2 -2 -2 -2]


run next line only if loaded

In [0]:
#vocab = vocab.tolist()

Bag-of-Words and Normalization
* remove really rare words
* downsample frequent words
* remove word gaps for words not in the vocabulary 
* flatten

In [15]:
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
print(compact)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=0)
# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
print(doc_ids)
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)
print(flattened)

[[1957 1956 1561 ...    0    0    0]
 [  76  157   84 ...    0    0    0]
 [ 462 1054   75 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]
[     0      1      2 ... 131422 131423 131424]
[1957 1956 1561 ... 2576  805 1222]


Pre-trained Word Vectors\
**TODO**: add pre-trained words when given

In [0]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
#!pip install gensim
assert flattened.min() >= 0
# Fill in the pretrained word vectors
n_dim = 30

import numpy as np
pretrained_filename= '/content/lda2vec/covid_embeddings/covid_19_w2v_embeds_30.model'


#vectors, s, f = corpus.compact_word_vectors(vocab, pretrained_filename,top=10027)
#model = Word2Vec.load(pretrained_filename)
#print(model.wv.syn0)

#print(vectors)

Save all files (uncomment for vectors if pre-trained included)

In [0]:
# Save all of the preprocessed files
pickle.dump(vocab, open('vocab.pkl', 'wb'))
pickle.dump(corpus, open('corpus.pkl', 'wb'))
np.save("flattened", flattened)
np.save("doc_ids", doc_ids)
np.save("pruned", pruned)
np.save("bow", bow)


In [19]:
np.save("vectors", vectors)

NameError: ignored

In [20]:
print(tokens.shape)
print(vocab)

(131425, 1439)
{0: 'agttgc', 1: 'gttgca', 2: 'ttgcag', 3: 'tgcaga', 4: 'gcagat', 5: 'cagata', 6: 'agataa', 7: 'gataat', 8: 'ataata', 9: 'taatat', 10: 'aatatg', 11: 'atatgg', 12: 'tatggt', 13: 'atggtg', 14: 'tggtgt', 15: 'ggtgtt', 16: 'gtgttt', 17: 'tgtttt', 18: 'gttttc', 19: 'ttttca', 20: 'tttcag', 21: 'ttcaga', 22: 'tcagat', 23: 'cagatg', 24: 'agatgc', 25: 'gatgct', 26: 'atgctg', 27: 'tgctgg', 28: 'gctggg', 29: 'ctggga', 30: 'tgggag', 31: 'gggagg', 32: 'ggaggt', 33: 'gaggtt', 34: 'aggtta', 35: 'ggttat', 36: 'gttatg', 37: 'ttatga', 38: 'tatgaa', 39: 'atgaaa', 40: 'tgaaac', 41: 'gaaaca', 42: 'aaacat', 43: 'aacata', 44: 'acatat', 45: 'catatt', 46: 'atattt', 47: 'tattta', 48: 'atttaa', 49: 'tttaaa', 50: 'ttaaaa', 51: 'taaaag', 52: 'aaaaga', 53: 'aaagat', 54: 'aagatt', 55: 'agatta', 56: 'gattaa', 57: 'attaag', 58: 'ttaagg', 59: 'taaggc', 60: 'aaggct', 61: 'aggctc', 62: 'ggctct', 63: 'gctctt', 64: 'ctcttt', 65: 'tctttg', 66: 'ctttga', 67: 'tttgac', 68: 'ttgacg', 69: 'tgacgt', 70: 'gacgta', 

In [21]:
%cd /content/lda2vec
#data_dir = os.getenv('data_dir', '../data/')
fn_vocab = 'vocab.pkl'
fn_corpus = 'corpus.pkl'
fn_flatnd = 'flattened.npy'
fn_docids = 'doc_ids.npy'
fn_vectors = 'vectors.npy'
vocab = pickle.load(open(fn_vocab, 'rb'))
corpus = pickle.load(open(fn_corpus, 'rb'))
flattened = np.load(fn_flatnd)
doc_ids = np.load(fn_docids)
vectors = np.load('/content/vectors.npy')

/content/lda2vec


FileNotFoundError: ignored

##Defining Model Parameters



>> Parameters 
1.   Number of documents
2.   Number of unique words
3.   Dirichlet Prior 
4.   Number of topics
5.   Batchsize
6.   power for negative sampling 
7.   flag to include pre-trained or not
8.   sampling temperature
9.   number of dimensions of word vectors
10.  number of tokens in each document
11.  token frequencies



In [0]:
# Model Parameters
# Number of documents
n_docs = doc_ids.max() + 1
# Number of unique words in the vocabulary
n_vocab = flattened.max() + 1
# 'Strength' of the dircihlet prior; 200.0 seems to work well
clambda = 200.0
# Number of topics to fit
n_topics = int(os.getenv('n_topics', 10))
batchsize = 32
# Power for neg sampling
power = float(os.getenv('power', 0.75))
# Intialize with pretrained word vectors
pretrained = bool(int(os.getenv('pretrained', True)))
# Sampling temperature
temperature = float(os.getenv('temperature', 1.0))
# Number of dimensions in a single word vector
n_units = int(os.getenv('n_units', 30))
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_vocab]
# How many tokens are in each document
doc_idx, lengths = np.unique(doc_ids, return_counts=True)
doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
doc_lengths[doc_idx] = lengths
# Count all token frequencies
tok_idx, freq = np.unique(flattened, return_counts=True)
term_frequency = np.zeros(n_vocab, dtype='int32')
term_frequency[tok_idx] = freq

In [23]:
for key in sorted(locals().keys()):
    val = locals()[key]
    if len(str(val)) < 100 and '<' not in str(val):
        print(key, val)

Out {12: 'LOAD IF ALREADY COMPUTED'}
_ LOAD IF ALREADY COMPUTED
_12 LOAD IF ALREADY COMPUTED
__ 
___ 
__doc__ if a model is already trained,
skip to the cell loading vocab,corpus,flattened and doc_ids
__loader__ None
__name__ __main__
__package__ None
__spec__ None
_dh ['/content', '/content/lda2vec']
_exit_code 0
_i10 print(sample[:100])
_i11 
mean_length = np.mean([len(t) for t in texts])
max_length = int(mean_length)
_i13 print(tokens[10,200:400])
_i14 #vocab = vocab.tolist()
_i18 np.save("vectors", vectors)
_i19 np.save("vectors", vectors)
_i20 print(tokens.shape)
print(vocab)
_i4 !python -m spacy download en
_i9 sample = np.random.choice(a=texts,size=100,replace=False)
_iii print(tokens.shape)
print(vocab)
_oh {12: 'LOAD IF ALREADY COMPUTED'}
batchsize 32
c 3321
clambda 200.0
doc_ids [ 0  0  0 ... 99 99 99]
filename /content/drive/My Drive/FAS_COMBINED.zip (Unzipped Files)/FAS_COMBINED/gor_hg3_content.txt
flattened [1957 1956 1561 ... 2576  805 1222]
fn_corpus corpus.pkl
fn_docids

# Training the LDA2vec Model

In [0]:
model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                n_samples=15, power=power, temperature=temperature)

Add pretrained data

In [0]:

pretrained = False

'''
if os.path.exists('/content/lda2vec.hdf5'):
    print("Reloading from saved")
    serializers.load_hdf5("/content/lda2vec.hdf5", model)
#print(model.sampler.W.data)
'''
if pretrained:     
    model.sampler.W.data[:, :] = vectors[:n_vocab, :]

Adam Optimizer

In [0]:
model.to_gpu()
optimizer = O.Adam()
optimizer.setup(model)
clip = chainer.optimizer.GradientClipping(5.0)
optimizer.add_hook(clip)

In [0]:
j = 0
epoch = 0
fraction = batchsize * 1.0 / flattened.shape[0]
progress = shelve.open('progress.shelve')

Training with cuda


*   Measure coherence between topics every 100 iterations
*   this is for only one epoch
*   save the model
*   backpropagation style 



In [28]:
n_epochs = 1
for epoch in range(n_epochs):
    data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
    top_words = print_top_words_per_topic(data)
    if j % 100 == 0 and j > 100:
        coherence = topic_coherence(top_words)
        for j in range(n_topics):
            print(j, coherence[(j, 'cv')])
        kw = dict(top_words=top_words, coherence=coherence, epoch=epoch)
        progress[str(epoch)] = pickle.dumps(kw)
    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    np.savez('topics.pyldavis', **data)
    print(epoch)
    for d, f in utils.chunks(batchsize, doc_ids, flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy())
        print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
               "P:{prior:1.3e} R:{rate:1.3e}")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        logs = dict(loss=float(l), epoch=epoch, j=j,
                    prior=float(prior.data), rate=rate)
        print(msg.format(**logs))
        j += 1
serializers.save_hdf5("lda2vec.hdf5", model)

Top words in topic 0 tcccta tcggtc gctgcg tgaggt ctttag ctcttt acatta tgatcg agggta atatat
Top words in topic 1 tgaaaa ttgtgc ttaaga agcttc ttagaa tccata gtacca tgttaa ctaccc ctctgc
Top words in topic 2 atcgaa attggg aatgtc gcctcc atccaa cttagg tcaaat catgcg ggtatc agtcgg
Top words in topic 3 ttttgt accgag ggaggg attgga taacaa atatct cccatg tggtgc gcgggg gctaaa
Top words in topic 4 caatgg aggttt ctatga ccaaca ggcaaa tttcgc aggcca agggta ttggta caggag
Top words in topic 5 gccctg gttatt agatcg gataac aggaat gctgga ctatgt ggtagg caagtg aagctt
Top words in topic 6 atagga tctctt tatcct atcttg ttaatg tgaggg acagtc agccat gtcaaa actagt
Top words in topic 7 agaaac agtctc tggcca ccaatt gcagaa gacata tttagc ggtctg aatgtg atctcg
Top words in topic 8 gtagtt atgaac agtgca tgtgta cagtgg caggag tgacag agattt tagagt taatct
Top words in topic 9 aaccta gcctaa ggtcta gcaatt agcggt tctcaa cagcct ccttct gcgatt gagagt
0
after partial fitting: 2292.934
J:00000 E:00000 L:2.293e+03 P:-2.076e+03 R:2.003e+00
aft

In [0]:
#!zip -r myzip.zip /content/lda2vec
#from google.colab import files
#files.download('myzip.zip')

In [30]:
npz = np.load(open('/content/lda2vec/topics.pyldavis.npz', 'rb'))
dat = {k: v for (k, v) in npz.items()}
dat['vocab'] = dat['vocab'].tolist()
print(dat['vocab'])

ValueError: ignored

Print top 10 words for each topic

In [31]:
top_n = 100
topic_to_topwords = {}
for j, topic_to_word in enumerate(data['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [data['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words

Topic 0 tcccta tcggtc gctgcg tgaggt ctttag ctcttt acatta tgatcg agggta atatat acctgt gtcaga atgccc atgcac tgttac caacgg aagcac aagaga atttaa tacgat ctagct gcgtgt tggacg ggttca ttatta cggcac aatttc ggacat ggtact cttggg tccctc gcctga ttcctc ttgagc ctcacc attgtg gggctc ctctct gtaggg tggggt gcactc ttgcac ttcccc atcaaa ctggtc ccagtt ttagac gaatgg gtaggt agcggg tagtga aacatt acacgc gattaa tccatc gtcatt gtaatg gttttc ttcata tggggg ggcgat tgaaac gtgtca tttcca ctgaaa tctggt gctctt ggggta gaccct tgggga tcgtct ttataa atacaa aatgac tagctg aggccc tgtacc tggatc ccaaaa ctactg tgttct cgttga agatca gaacac ctgcta gtaggc acgatg agcctc agacgg aaattc aaaagc acggta acagga gggatc ggataa tgcagc cattaa gtaccc attacc actatc
Topic 1 tgaaaa ttgtgc ttaaga agcttc ttagaa tccata gtacca tgttaa ctaccc ctctgc tcaaga gactcc gtagat agaccc gaggag aggaag aaccgt aacaag tctgcc gacaaa catatc attatt gtattc ggtcag tgagga ccaagg ggctga taacca agcata cccttt agtgtg taacta ttattc gatcat ggctaa tatgca ggtatc ttgttg ttgatg gagtgg tggc

In [0]:
pickle.dump(model.mixture.weights.W.data, open('weights.pkl', 'wb'))
pickle.dump(model.mixture.factors.W.data, open('factors.pkl', 'wb'))
pickle.dump(model.sampler.W.data, open('sampler.pkl', 'wb'))


## Visualize Topics 
>> lower λ to get rare words appearing only on selected topics

In [0]:
!pip install pyLDAvis
import pyLDAvis
pyLDAvis.enable_notebook()
#warnings.filterwarnings('ignore')
import math

#dat['topic_term_dists'] = dat['topic_term_dists'] / dat['topic_term_dists'].sum(axis=1)[:, None]
prepared_data = pyLDAvis.prepare(data['topic_term_dists'], data['doc_topic_dists'], 
                                 data['doc_lengths'] * 1.0, data['vocab'], data['term_frequency'] * 1.0, mds='tsne')

In [36]:
pyLDAvis.display(prepared_data)